In [1]:
import torch
import random
import torch.optim as optim
import torch.nn as nn
from torch.nn import init

In [2]:
#训练集
x0 = torch.normal(2*torch.ones(7000,200),1)
y0 = torch.zeros(7000,1)
x1 = torch.normal(-2*torch.ones(7000,200),1)
y1 = torch.ones(7000,1)
x_t = torch.cat((x0, x1), 0).type(torch.FloatTensor)
y_t = torch.cat((y0, y1), 0).type(torch.FloatTensor)
#测试集
x0_test = torch.normal(1*torch.ones(3000,200),1)
y0_test = torch.zeros(3000,1)
x1_test = torch.normal(-1*torch.ones(3000,200),1)
y1_test = torch.ones(3000,1)
x_t_test = torch.cat((x0_test, x1_test), 0)
y_t_test = torch.cat((y0_test, y1_test), 0)

In [3]:
#读取数据
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)])
        yield features.index_select(0, j), labels.index_select(0, j)

In [4]:
#交叉熵损失
loss = torch.nn.BCELoss()

In [5]:
class FlattenLayer(torch.nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x):
        return x.view(x.shape[0], -1)
net = nn.Sequential(
    FlattenLayer(),
    nn.Linear(200,128),
    nn.ReLU(),
    nn.Linear(128,1),
)

In [6]:
for params in net.parameters():
    init.normal_(params,mean=0,std=0.01)

In [7]:
#K折
def get_kfold_data(k, i, X, y):
    fold_size = X.shape[0] // k
    val_start = i * fold_size
    if i != k - 1:
        val_end = (i + 1) * fold_size
        X_valid, y_valid = X[val_start:val_end], y[val_start:val_end]
        X_train = torch.cat((X[0:val_start], X[val_end:]), dim = 0)
        y_train = torch.cat((y[0:val_start], y[val_end:]), dim = 0)
    else:
        X_valid, y_valid = X[val_start:], y[val_start:]
        X_train = X[0:val_start]
        y_train = y[0:val_start]
    return X_train, y_train, X_valid, y_valid

In [8]:
def k_fold(k, X_train, y_train):
    train_loss_sum, valid_loss_sum = 0, 0
    train_acc_sum, valid_acc_sum = 0, 0
    for i in range(k):
        print('第', i + 1, '折验证结果')
        data = get_kfold_data(k, i, X_train, y_train)
        for params in net.parameters():
            init.normal_(params, mean=0, std=0.01)
        train_loss, val_loss, train_acc, val_acc = train(*data)
        print('train loss:{:.4f}, train accuracy:{:.3f}'.format(train_loss, train_acc))
        print('valid loss:{:.4f}, valid accuracy:{:.3f}'.format(val_loss, val_acc))
        train_loss_sum += train_loss
        valid_loss_sum += val_loss
        train_acc_sum += train_acc
        valid_acc_sum += val_acc
    print('\n', '最终10折交叉验证结果：')
    print('ave_train_loss:{:.4f}, ave_train_acc:{:.3f}'.format(train_loss_sum / k, train_acc_sum / k))
    print('ave_valid_loss:{:.4f}, ave_valid_acc:{:.3f}'.format(valid_loss_sum / k, valid_acc_sum / k))
    return

In [9]:
#优化
learn_rate = 0.001
optimizer = optim.SGD(net.parameters(), lr=learn_rate)

In [10]:
def evaluate(x_test, y_test, num):
    acc_sum = 0
    for X, Y in data_iter(1, x_test, y_test):
        if (net(X) >= 0.5 and Y == 1) or (net(X) < 0.5 and Y == 0):
            acc_sum += 1
    return acc_sum/num

In [11]:
#参数
batch_size = 128
num_epochs = 10
xzhou = []
y_train_loss = []
y_train_acc = []
y_test_loss = []
y_test_acc = []
#训练
def train(x_t, y_t, x_t_test, y_t_test):
    y_train_loss, y_test_loss, y_train_acc, y_test_acc = [], [], [], []
    for epoch in range(num_epochs):
        for X, Y in data_iter(batch_size, x_t, y_t):
            l = loss(torch.sigmoid(net(X)), Y).sum()
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            optimizer.step()
        train_l = loss(torch.sigmoid(net(x_t)), y_t)
        xzhou.append(epoch + 1)
        y_train_loss.append(train_l.mean().item())
        y_train_acc.append(evaluate(x_t, y_t, 12600))
        train_l = loss(torch.sigmoid(net(x_t_test)), y_t_test)
        y_test_loss.append(train_l.mean().item())
        y_test_acc.append(evaluate(x_t_test, y_t_test, 1400))
    return y_train_loss[num_epochs-1], y_test_loss[num_epochs-1], y_train_acc[num_epochs-1], y_test_acc[num_epochs-1]

In [12]:
k_fold(10, x_t, y_t)

第 1 折验证结果
train loss:0.0149, train accuracy:1.000
valid loss:0.0178, valid accuracy:1.000
第 2 折验证结果
train loss:0.0151, train accuracy:1.000
valid loss:0.0179, valid accuracy:1.000
第 3 折验证结果
train loss:0.0148, train accuracy:1.000
valid loss:0.0174, valid accuracy:1.000
第 4 折验证结果
train loss:0.0155, train accuracy:1.000
valid loss:0.0189, valid accuracy:1.000
第 5 折验证结果
train loss:0.0155, train accuracy:1.000
valid loss:0.0194, valid accuracy:1.000
第 6 折验证结果
train loss:0.0148, train accuracy:1.000
valid loss:0.0178, valid accuracy:1.000
第 7 折验证结果
train loss:0.0152, train accuracy:1.000
valid loss:0.0189, valid accuracy:1.000
第 8 折验证结果
train loss:0.0150, train accuracy:1.000
valid loss:0.0175, valid accuracy:1.000
第 9 折验证结果
train loss:0.0153, train accuracy:1.000
valid loss:0.0194, valid accuracy:1.000
第 10 折验证结果
train loss:0.0151, train accuracy:1.000
valid loss:0.0179, valid accuracy:1.000

 最终10折交叉验证结果：
ave_train_loss:0.0151, ave_train_acc:1.000
ave_valid_loss:0.0183, ave_valid_acc:1.00